# Figures: uncertainty quantification for measurement errors

## Imports

In [ ]:
import dolfin  # https://fenicsproject.org
import numpy
import multiprocessing
import subprocess
import time
import copy

import pandas as pd

import os
import helpers
import compute_disp
import matplotlib.pyplot as plt

import dolfin_mech as dmech

import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger('FFC').setLevel(logging.WARNING)


## Parameters

#### Material and loading parameters

In [ ]:
#### parameters should be given as dictionary; additionally the parameter alpha_fibrose_i should increase as i increases

# ## Figure 4
# parameters_to_identify = {"alpha":0.09}

# ## Figure 5
# parameters_to_identify = {"pi":-2}

## Figure 6 - top row
parameters_to_identify = {"alpha_healthy":0.09, "pi":-2}

## Figure  7 - top row
# parameters_to_identify = {"alpha_healthy":0.09, "alpha_fibrose":0.9}

# ## Figure  8 - top row
# parameters_to_identify = {"alpha_fibrose":0.9, "pi":-2}

# ## Figure  9 - top row
# parameters_to_identify = {"alpha_fibrose_1":0.9, "alpha_fibrose_2":0.67, "pi":-2}

# ## Figure 10 / 12 - top row 
# parameters_to_identify = {"alpha_fibrose_1":1.1, "alpha_fibrose_2":0.9, "alpha_fibrose_3":0.67, "pi":-2}

# ## Figure 11 - top row 
# parameters_to_identify = {"alpha_healthy": 0.09, "alpha_fibrose": 0.67, "pi":-2}


nb_parameters = len(parameters_to_identify)

reference_value = []
param_names = []

for param_name, param_value in parameters_to_identify.items():
    reference_value.append(param_value)
    param_names.append(param_name)


### default parameter values 
alpha, alpha_healthy, alpha_fibrose, alpha_fibrose_1, alpha_fibrose_2, alpha_fibrose_3 = 0.09, 0.09, 0.67, 0.67, 0.9, 1.1 ### reference stiffnesses in kPa
gamma = 0.5 ### [-]
c1, c2 = 0.2, 0.4 ### in kPa
pe, pi = -0.5, -2 ### end-exhalation and end-inhalation pleural pressures, in kPa

#### Computation parameters

In [ ]:
positions = ["Supine", "Prone", "Supine+Prone"] ### supine, prone, prone + supine

noise_lst = [0., 1/20, 1/10, 1/5, 1/2.5]
SNR_lst = [ (2/noise_lst[i+1] if noise_lst[i] == 0 else 1/noise_lst[i]) for i in range(len(noise_lst))]


### Geometry

In [ ]:
### generic mesh is stored in Data folder
cube_params = {"path_and_mesh_name": "./Data/test.xdmf"}

### Loading

In [ ]:
### unloading problem

### generic unloading problem, no gravity is applied
load_params_unloading_generic = {"type":"p_boundary_condition0", "f":0, "P0" : float(pe)} 

### loading problem

### generic end-exhalation configuration in prone and in supine positions
load_params_loading_prone = {"type":"p_boundary_condition", "f": -9.81e3, "P0" : float(pe)}
load_params_loading_supine = {"type":"p_boundary_condition", "f": 9.81e3, "P0" : float(pe)}


### Directory names

In [ ]:
current_directory = os.getcwd()

directory_prone = current_directory + "/prone" ### storing the results created for prone position
directory_supine = current_directory + "/supine" ### storing the results created for supine position

### creating associated directories, if do not already exist

try:
    os.mkdir(directory_prone)
except:
    pass

try:
    os.mkdir(directory_supine)
except:
    pass

### Lists and dictionaries

In [ ]:
def reinitialize_lsts():

    storing_processes = {} ### storing processes launched in parallel

    results = {} ### storing the results of the estimation, for each noise level
    storing_values_for_convergence_check = {} ### storing the results for convergence checks

    ### initializing lists in results dict
    results['noise'] = []

    for param_name in param_names:
        lst_name = "ini_"+str(param_name)
        results[lst_name] = []
        results[param_name] = []

    for noise in noise_lst:
        storing_values_for_convergence_check[str(noise)] = []
        for i in range(nb_parameters):
            storing_values_for_convergence_check[str(noise)].append([])

    return(results, storing_values_for_convergence_check, storing_processes)

## Write reference end-exhalation configurations -prone and supine- from Zygot

In [ ]:
### creating mesh for end-exhalation prone and supine
mesh_prone = dolfin.Mesh()
mesh_supine = dolfin.Mesh()
mesh_name = str(cube_params["path_and_mesh_name"])
dolfin.XDMFFile(mesh_name).read(mesh_prone)
dolfin.XDMFFile(mesh_name).read(mesh_supine)

### generic configurations, computed with one zone for the sake of simplicity
parameters = {"alpha": alpha, "gamma":gamma, "c1":c1, "c2":c2, "kappa":1e2, "eta":1e-5}
mat_params={"scaling": "linear", "parameters": parameters}

### computing unloaded configuration from generic Zygot
U_unloading, phis_unloading, dV_unloading = dmech.run_RivlinCube_PoroHyperelasticity(
    dim = 3,
    inverse = 1,
    cube_params = cube_params,
    porosity_params = {"type": "mesh_function_random_xml"},
    get_results = 1,
    mat_params = mat_params,
    step_params = {"dt_min":1e-4},
    load_params = load_params_unloading_generic,
    res_basename = "generic_unloaded",
    inertia_params = {"applied":True},
    plot_curves = 0,
    verbose =1 )

#### redefining porosity field in the unloaded configuration so it is physiological
phisref_imposed = list(numpy.random.uniform(low = 0.4, high = 0.6, size = len(phis_unloading)))

#### computing end-exhalation configuration prone position
Uprone, phisprone, dVprone = dmech.run_RivlinCube_PoroHyperelasticity(
    dim = 3,
    inverse = 0,
    porosity_params = {"type":"function_xml_from_array", "val":phisref_imposed},
    cube_params = cube_params,
    mat_params = mat_params,
    step_params = {"dt_ini": 0.125, "dt_min": 1e-4},
    load_params = load_params_loading_prone,
    res_basename = directory_prone+"/prone",
    move_params = {"move": True, "U": U_unloading}, ### applying the displacement field from generic end-exhalation configuration to unloaded configuration
    inertia_params={"applied":True},
    get_results = 1,
    plot_curves = 0,
    verbose = 1)

helpers.write_porosity(porosity_field = phisprone, n_cells = len(mesh_prone.cells()), filepath = directory_prone + "prone-poro.xml")

### getting displacement field from generic end-exhalation to prone end-exhalation configuration
Uexhal_prone = U_unloading.copy(deepcopy=True)
Uexhal_prone.vector()[:] +=  Uprone.vector()[:]
dolfin.ALE.move(mesh_prone, Uexhal_prone)

### writing mesh prone
xdmf_file_mesh = dolfin.XDMFFile("prone/Zygotprone.xdmf")
xdmf_file_mesh.write(mesh_prone)
xdmf_file_mesh.close()


#### computing end-exhalation configuration supine position
Usupine, phissupine, dVsupine = dmech.run_RivlinCube_PoroHyperelasticity(
    dim = 3,
    inverse = 0,
    porosity_params = {"type":"function_xml_from_array", "val":phisref_imposed},
    cube_params = cube_params,
    mat_params = mat_params,
    step_params = {"dt_ini": 0.125, "dt_min": 1e-4},
    load_params = load_params_loading_supine,
    res_basename = directory_supine+"/supine",
    move_params = {"move": True, "U": U_unloading},  ### applying the displacement field from generic end-exhalation configuration to unloaded configuration
    inertia_params={"applied": True},
    get_results = 1,
    plot_curves=0,
    verbose=1)

helpers.write_porosity(porosity_field = phissupine, n_cells = len(mesh_supine.cells()), filepath = directory_supine + "/supine-poro.xml")

### getting displacement field from generic end-exhalation to prone end-exhalation configuration
Uexhal_supine = U_unloading.copy(deepcopy=True)
Uexhal_supine.vector()[:] +=  Usupine.vector()[:]
dolfin.ALE.move(mesh_supine, Uexhal_supine)

### writing mesh supine
xdmf_file_mesh = dolfin.XDMFFile(directory_supine+"/Zygotsupine.xdmf")
xdmf_file_mesh.write(mesh_supine)
xdmf_file_mesh.close()

## Plotting

In [ ]:
def plotting_and_writing(positions = ['Supine'], parameters_to_identify={}, results = {}):
    
    for parameter_name, value in parameters_to_identify.items():
        fig, ax = plt.subplots()
        ax.cla()
        ### plotting parameters
        plt.rc("xtick", labelsize=18)
        plt.rc("ytick", labelsize=18)
        plt.rc("legend", fontsize=16)
        plt.ylim([-100, 100])
        plt.xlabel("Signal to Noise Ratio (SNR)", fontsize=14)
        plt.ylabel("Estimation error (%)", fontsize=14)
        color_lst=['royalblue', 'firebrick', 'forestgreen'] ### different color for each position
        color_lst_lines=['royalblue', 'firebrick', 'darkgreen']
        alpha_lst=[0.6, 0.45, 0.6] ### transparencies to be able 
        label_lst = positions 
        for position in positions:
            ### reorganize data 
            results_position = results[position]
            frame = pd.DataFrame(results_position)
            sorted_frame = frame.sort_values(by="noise")
            parametrization_name = ''
        
            stats_results = sorted_frame.groupby("noise")[str(parameter_name)].agg(['mean', 'std'])
            stats_results['mean_'+str(parameter_name)] = stats_results['mean']
            stats_results['mean_plus_std'+str(parameter_name)] = stats_results['mean'] + stats_results['std']
            stats_results['mean_minus_std'+str(parameter_name)] = stats_results['mean'] - stats_results['std']
            parametrization_name += parameter_name

            plt.plot(SNR_lst, stats_results['mean_'+str(parameter_name)], color=color_lst_lines[0], label=label_lst[0])
            ax.fill_between(SNR_lst, stats_results['mean_minus_std'+str(parameter_name)], stats_results['mean_plus_std'+str(parameter_name)], alpha=alpha_lst[0], color=color_lst[0])
            color_lst=color_lst[1:]
            label_lst=label_lst[1:]
            alpha_lst=alpha_lst[1:]
            color_lst_lines=color_lst_lines[1:]
        plt.xlim([2.5, 44])
        plt.gca().set_xscale('log')
        ax.errorbar(SNR_lst, len(SNR_lst)*[0], yerr=30, linewidth=1, markersize=10, color='black', fmt='x', capsize=5, label="Initial distribution")
        plt.legend()
        plt.grid()
        plt.show()
        plt.savefig("parametrization="+parametrization_name+"_identification_parameter="+str(parameter_name)+"_position="+str(position)+"_impact_measurement_errors.pdf", bbox_inches='tight')


## Helpers

In [ ]:
results_all_positions = {}
for position in positions:
    print("position", position)
    distribution_converged = ['no'] * len(noise_lst) ### is the distribution converged for each noise level?

    results, storing_values_for_convergence_check, storing_processes = reinitialize_lsts()

    ########### computation of reference displacement field
    if position=='Supine+Prone':
        
        ### computing displacement field in supine position
        Umeas_supine, Vmeas_supine = compute_disp.compute_disp(gravity_type = 'Supine', parameters_to_identify = parameters_to_identify, noise='', dirpath = current_directory)
        V0_supine = dolfin.assemble(dolfin.Constant(1)*Vmeas_supine)
        Umeas_norm_supine = (dolfin.assemble(dolfin.inner(Umeas_supine, Umeas_supine)*Vmeas_supine)/2/V0_supine)**(1/2)

        ### writing displacement field supine position
        file_supine = dolfin.File("refSupine/displacement_exhal_to_inhal.xml")
        file_supine << Umeas_supine
        file_supine.close()

        ### computing displacement field in prone position
        Umeas_prone, Vmeas_prone = compute_disp.compute_disp(gravity_plus = 'Prone', parameters_to_identify = parameters_to_identify, noise='', dirpath = current_directory)
        V0_prone = dolfin.assemble(dolfin.Constant(1)*Vmeas_prone)
        Umeas_norm_prone = (dolfin.assemble(dolfin.inner(Umeas_prone, Umeas_prone)*Vmeas_prone)/2/V0_prone)**(1/2)

        ### writing displacement field prone position
        file_prone = dolfin.File("refProne/displacement_exhal_to_inhal.xml")
        file_prone << Umeas_prone

    else :
        Umeas, Vmeas = compute_disp.compute_disp(position = position, parameters_to_identify = parameters_to_identify, noise = '', dirpath = current_directory)
        V0 = dolfin.assemble(dolfin.Constant(1)*Vmeas)
        Umeas_norm = (dolfin.assemble(dolfin.inner(Umeas, Umeas)*Vmeas)/2/V0)**(1/2)

        ### writing displacement field in prone or supine (depending on the case investigated) position
        file_prone = dolfin.File("ref"+str(position)+"/displacement_exhal_to_inhal.xml")
        file_prone << Umeas


    number_cpus = multiprocessing.cpu_count()

    min_iterations = number_cpus // len(noise_lst) ### arbitrary, to change depending on the resource available for example

    converged = False ### convergence of the error distributions
    while not converged:
        converged = all(convergence_status == "converged" for convergence_status in distribution_converged)
        over = False
        for i in range(len(noise_lst)):
            if distribution_converged[i] == 'converged':
                storing_processes.pop(str(noise_lst[i]), None)
            elif distribution_converged[i] == 'no':
                storing_processes[str(noise_lst[i])] = []

        for noise, lst in storing_processes.items():
            if lst == []:
                for iteration in range(0, min_iterations):
                    ini_calculation = []
                    for param, param_value in parameters_to_identify.items():
                        ini_calculation.append(float(numpy.random.normal(loc = param_value, scale = abs(0.3*param_value), size = 1)[0]))
                    process = subprocess.Popen(["python",  "-W", "%s" %"ignore", "./minimization.py", "--position", "%s" %position, "--noise_level", "%s" %noise, "--parameters_to_identify", "%s" %parameters_to_identify, "--iteration", "%s" %iteration, "--dirpath", "%s" %current_directory, "--initialization", "%s" %ini_calculation], stdout=subprocess.PIPE )
                    storing_processes[str(noise)].append(process)
        while not over:
            time.sleep(1)
            for noise, lst in storing_processes.items():
                noise = float(noise)
                over = helpers.checking_if_processes_converged(storing_processes[str(noise)])
                if over:
                    for process in storing_processes[str(noise)]:
                        try:
                            out = process.communicate()[0]
                            out = out.decode("utf-8").split()
                        except:
                            pass                        
                        solution = {}
                        if out != []: #### out is [] if the minimization did not converge
                            results['noise'].append(noise)
                            for i in range(0, nb_parameters): ### first 
                                lst_name = "ini_"+str(param_names[i])
                                results[lst_name].append(((float(out[i])-reference_value[i]))/reference_value[i]*100)
                            for k in range (nb_parameters, 2*nb_parameters):
                                i = k - nb_parameters
                                results[param_names[i]].append((float(out[k]) - reference_value[i])/reference_value[i]*100)
                                storing_values_for_convergence_check[str(noise)][i].append(float(out[k]))
                    if len(storing_values_for_convergence_check[str(noise)][0]) > min_iterations+1:
                        converged, crit = helpers.checking_if_converged(storing_values_for_convergence_check[str(noise)], len(storing_processes[str(noise)]), tol=1e-2)
                    if converged:
                        distribution_converged[noise_lst.index(noise)] = 'converged'
                    else:
                        distribution_converged[noise_lst.index(noise)] = 'no'
    results_all_positions[position] = results
    df = pd.DataFrame(results)
    df_sorted = df.sort_values(by="noise")
    myfile= open("./results_estimation-position="+str(position), 'w')
    myfile.write(df_sorted.to_string(index=False))
    myfile.close()
plotting_and_writing(positions = positions, parameters_to_identify = parameters_to_identify, results = results_all_positions)